In [27]:
import pandas as pd
import utils 
import plotly.graph_objects as go
import insturment

In [28]:
pair = "GBP_JPY"
granularity = "H1"
ma_list =[16,64]
i_pair = insturment.Instrument.get_instrument_by_name(pair)

In [29]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
# df.head()
# df.describe()
none_columns = ['time', 'volume']
mod_columns = [x for x in df.columns if x not in none_columns ]
df[mod_columns] = df[mod_columns].apply(pd.to_numeric)

FileNotFoundError: [Errno 2] No such file or directory: 'his_data/GBY_JPY_H1.pkl'

In [15]:
df_ma = df[['time','mid_o','mid_h','mid_l','mid_c']].copy()
# df_ma.head()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True,inplace=True )

In [16]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2024-02-08T14:00:00.000000000Z,1.07442,1.07675,1.07436,1.07651,1.077411,1.076045
1,2024-02-08T15:00:00.000000000Z,1.07652,1.07678,1.07514,1.07614,1.077322,1.076076
2,2024-02-08T16:00:00.000000000Z,1.07614,1.07742,1.07604,1.07733,1.077293,1.076122
3,2024-02-08T17:00:00.000000000Z,1.07735,1.07767,1.07715,1.07735,1.077261,1.076166
4,2024-02-08T18:00:00.000000000Z,1.07736,1.07828,1.07732,1.07767,1.077222,1.076214


In [17]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

the top indcates a buy if row.DIFF >= 0 and row.DIFF_PREV < 0:
 and bottom one is a sell  if row.DIFF <= 0 and row.DIFF_PREV > 0:

In [18]:
df_ma['DIFF'] =df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE != 0].copy()

making a new column, taking exsitsing dataFrame and useing Apply{applies the the whole date frame} axis 1 mean we are working row by row, the result of the fuction is stored in the 'IS_TRADE' column.

shifts the colums down by one ...-1 will shift it up 

a fuction to calculate the a value for a nnew column

In [24]:
df_trades


,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN
54,2024-02-12T20:00:00.000000000Z,1.07695,1.07750,1.07695,1.07743,1.077698,1.077756,-0.000058,0.000034,-1,-7.9,7.9
119,2024-02-15T13:00:00.000000000Z,1.07448,1.07730,1.07428,1.07664,1.073391,1.073214,0.000177,-0.000074,1,50.3,50.3
274,2024-02-26T00:00:00.000000000Z,1.08190,1.08204,1.08161,1.08167,1.082423,1.082425,-0.000002,0.000079,-1,41.4,-41.4
287,2024-02-26T13:00:00.000000000Z,1.08497,1.08590,1.08484,1.08581,1.083079,1.082911,0.000169,-0.000005,1,-34.7,-34.7
329,2024-02-28T07:00:00.000000000Z,1.08221,1.08250,1.08166,1.08234,1.083991,1.084050,-0.000059,0.000092,-1,17.5,-17.5
...,...,...,...,...,...,...,...,...,...,...,...,...
3723,2024-09-12T18:00:00.000000000Z,1.10563,1.10626,1.10512,1.10608,1.102747,1.102731,0.000016,-0.000262,1,53.2,53.2
3824,2024-09-18T23:00:00.000000000Z,1.11220,1.11220,1.11112,1.11140,1.112438,1.112487,-0.000049,0.000057,-1,44.0,-44.0
3836,2024-09-19T11:00:00.000000000Z,1.11706,1.11762,1.11520,1.11580,1.112919,1.112650,0.000269,-0.000013,1,-44.9,-44.9
3883,2024-09-23T10:00:00.000000000Z,1.11063,1.11194,1.11054,1.11131,1.114686,1.114728,-0.000041,0.000228,-1,66.6,-66.6


In [23]:
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"]  * df_trades["IS_TRADE"]

In [25]:
df_trades["GAIN"].sum()

np.float64(-377.9999999999961)

In [9]:
df_plot = df_ma.iloc[50:3999].copy()

In [10]:
df_plot.shape


(3886, 10)

In [11]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x= df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
     line=dict(width=1),opacity=1,
     increasing_fillcolor='#24A06B',
     decreasing_fillcolor='#CC2E3C',
     increasing_line_color='#2EC886',
     decreasing_line_color='#FF3A4C'))
     
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time,
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col))

fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10, color="#e1e1e1"),
    paper_bgcolor= "#1e1e1e",
    plot_bgcolor="#1e1e1e")

fig.update_xaxes(gridcolor="#1f292f",showgrid=True,fixedrange=True,rangeslider=dict(visible=False))
fig.update_yaxes(gridcolor="#1f292f",showgrid=True,fixedrange=True)

fig.show()